# Model Processing

## Imports & General Settings 

In [1]:
import unittest
import os
import sys
import time
import pathlib

import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToPILImage
import PIL
from tqdm.notebook import trange, tqdm
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, average_precision_score, roc_curve, roc_auc_score

from matplotlib import pyplot

# Our imports
from data import WaveletTransform, AFECGDataset
import dsp
from model.blocks import ConvNet, BRNN, SoftmaxAttention
from model.baseline import Baseline
from training import train, test
import utils


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
test = unittest.TestCase()
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


## Dataset creation

In [3]:
dataset_name = 'afdb'
dataset = AFECGDataset(dataset_name, '../data/files/')

In [4]:
dataset.load()

36it [00:00, 358.35it/s]

Preparing 1397 samples


1397it [00:05, 235.85it/s]


Elapsed time: 5925.168991088867 ms
Skipped 1397 files which had a backup


In [ ]:
data_af, label_af = dataset[0]
data_nsr, label_nsr = dataset[1]

t = data_nsr[0]
utils.show_spectrogram(t)

In [ ]:
images_per_sample = 20
total_data_size = len(dataset)
print("Total data size: ", total_data_size)
print("Patients with AF: ", dataset.labels.sum().item())

In [ ]:
# data = [dataset[i][0] for i in range(total_data_size)]
# labels = [dataset[i][1] for i in range(total_data_size)]

### Example of one ECG sample

In [ ]:
# samples, label = data[0], labels[0]
# print('P-signal: ', samples)
# print('Has AF: ', 'Yes' if label == 1 else 'No')

In [ ]:
# to_wavelet = WaveletTransform(wavelet.Morlet(6), resample=20)
# t = to_wavelet(data[0][0])
# image_test = (t * 100 * 255).int() # Simple visualization test
# transforms.ToPILImage()(image_test).show()

##  Wavelet Transform

In [ ]:
# Total data size is 1397
# You can choose the data size 
data_size = len(dataset)

In [ ]:
# x0 = x_train[0][0].float()
# encoder_cnn = ConvNet((375, 20))

# display(x0.unsqueeze(0).shape)
# h = encoder_cnn(x0.unsqueeze(0))
# print(h.shape)

# test.assertEqual(h.dim(), 2)
# test.assertSequenceEqual(h.shape, (1, 50))

## BRNN

In [ ]:
display(BRNN(50, 50, images_per_sample))

## Attention

Notations:

* $Y = \left[ y_1, \ldots, y_T \right]$ – the input matrix of size $\left( N \times T \right)$, where $N$ is the number of features in a single output vector of the BRNN

* $w_\mathrm{att}$ – The parameters of the attention model, of size $\left( N \times 1 \right)$, where $N$ is the number of features in a single output vector of the BRNN

* $\alpha$ – The attention weights, given as $\alpha = \mathrm{softmax} \left( w_\mathrm{att}^T Y \right)$. This is an element-wise softmax, where the output size of $\alpha$ is $\left( 1 \times T \right)$

* $h_\mathrm{att}$ – Output of the attention mechanism, given by $h_\mathrm{att} = Y \alpha^T$, of size $\left( N \times 1 \right)$, i.e. a vector of $N$ features.

In [ ]:
SoftmaxAttention(100)

## Training

In [ ]:
model = Baseline()

### CNN

In [ ]:
data1, label1 = dataset[0]
utils.show_spectrogram(data1[0])

In [13]:
heldout = int(len(dataset) * 0.2)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - heldout, heldout])

model = nn.Sequential(
    ConvNet(size=(375, 20)),
    nn.Linear(50, 2)
)
config = dict(
    num_workers=8,
    batch_size=90,
    learning_rate=0.001,
    weight_decay=0.01,
    
    num_epochs=200,

#     num_epochs=200,
    is_notebook=True
)

train(model, train_dataset, config)

torch.Size([90, 20, 375])
Loss:  tensor(0.6827, grad_fn=<NllLossBackward>)
Output: tensor([[-0.0524, -0.0870],
        [-0.0494, -0.0965],
        [-0.0521, -0.0965],
        [-0.0475, -0.0901],
        [-0.0344, -0.0886],
        [-0.0489, -0.0961],
        [-0.0342, -0.0994],
        [-0.0527, -0.0960],
        [-0.0449, -0.1113],
        [-0.0332, -0.0958],
        [-0.0374, -0.1049],
        [-0.0506, -0.0961],
        [-0.0495, -0.0823],
        [-0.0415, -0.0967],
        [-0.0545, -0.1032],
        [-0.0478, -0.0911],
        [-0.0516, -0.0960],
        [-0.0472, -0.1069],
        [-0.0602, -0.0915],
        [-0.0448, -0.0836],
        [-0.0465, -0.1102],
        [-0.0451, -0.1063],
        [-0.0509, -0.0968],
        [-0.0606, -0.1057],
        [-0.0450, -0.0968],
        [-0.0423, -0.0958],
        [-0.0494, -0.0953],
        [-0.0401, -0.0594],
        [-0.0492, -0.1093],
        [-0.0495, -0.0908],
        [-0.0473, -0.0848],
        [-0.0401, -0.1038],
        [-0.0468, -0.

Output: tensor([[ 0.2912, -0.7031],
        [ 0.2938, -0.6840],
        [ 0.2448, -0.6383],
        [ 0.2363, -0.6187],
        [ 0.1519, -0.4346],
        [ 0.2376, -0.6190],
        [ 0.2075, -0.5687],
        [ 0.2971, -0.7233],
        [ 0.2971, -0.7134],
        [ 0.2350, -0.6182],
        [ 0.2592, -0.6635],
        [ 0.2383, -0.6124],
        [ 0.3148, -0.7518],
        [ 0.2103, -0.5760],
        [ 0.2443, -0.6327],
        [ 0.2942, -0.7191],
        [ 0.3120, -0.7140],
        [ 0.2896, -0.7058],
        [ 0.2604, -0.6533],
        [ 0.2964, -0.7233],
        [ 0.3028, -0.7210],
        [ 0.2560, -0.6521],
        [ 0.2415, -0.6330],
        [ 0.2164, -0.5534],
        [ 0.2960, -0.6914],
        [ 0.2841, -0.7003],
        [ 0.1764, -0.4752],
        [ 0.1178, -0.3867],
        [ 0.2153, -0.5781],
        [ 0.3041, -0.7343],
        [ 0.2797, -0.6950],
        [ 0.3107, -0.7447],
        [ 0.2534, -0.6528],
        [ 0.1915, -0.5233],
        [ 0.2470, -0.6411],
        [ 0.

Output: tensor([[ 0.3973, -0.8354],
        [ 0.3627, -0.7891],
        [ 0.3894, -0.8320],
        [ 0.2799, -0.6466],
        [ 0.4035, -0.8532],
        [ 0.3480, -0.7640],
        [ 0.3643, -0.7898],
        [ 0.3219, -0.7166],
        [ 0.3471, -0.7633],
        [ 0.4040, -0.8540],
        [ 0.3702, -0.7975],
        [ 0.4193, -0.8747],
        [ 0.3877, -0.8187],
        [ 0.3851, -0.8176],
        [ 0.3876, -0.8273],
        [ 0.3297, -0.7377],
        [ 0.3307, -0.7326],
        [ 0.3351, -0.7421],
        [ 0.2443, -0.5952],
        [ 0.3549, -0.7692],
        [ 0.3811, -0.8071],
        [ 0.2762, -0.6433],
        [ 0.3523, -0.7684],
        [ 0.3333, -0.7412],
        [ 0.3916, -0.8344],
        [ 0.3249, -0.7219],
        [ 0.3622, -0.7855],
        [ 0.3876, -0.8283],
        [ 0.2387, -0.5844],
        [ 0.2278, -0.5612],
        [ 0.2819, -0.6536],
        [ 0.3956, -0.8364],
        [ 0.3882, -0.8192],
        [ 0.3959, -0.8406],
        [ 0.4025, -0.8511],
        [ 0.

torch.Size([90, 20, 375])
Loss:  tensor(0.5859, grad_fn=<NllLossBackward>)
Output: tensor([[ 0.1767, -0.4596],
        [ 0.2510, -0.5666],
        [ 0.2802, -0.6115],
        [ 0.1840, -0.4687],
        [ 0.1824, -0.4586],
        [ 0.2729, -0.6004],
        [ 0.2631, -0.5843],
        [ 0.2788, -0.6099],
        [ 0.3257, -0.6723],
        [ 0.2659, -0.5897],
        [ 0.3028, -0.6434],
        [ 0.2790, -0.6100],
        [ 0.1653, -0.4351],
        [ 0.2164, -0.5194],
        [ 0.2462, -0.5596],
        [ 0.2294, -0.5374],
        [ 0.2752, -0.6047],
        [ 0.2798, -0.6115],
        [ 0.2115, -0.5076],
        [ 0.2528, -0.5726],
        [ 0.3187, -0.6648],
        [ 0.2688, -0.5941],
        [ 0.2791, -0.6094],
        [ 0.1596, -0.4272],
        [ 0.2405, -0.5473],
        [ 0.2382, -0.5445],
        [ 0.2560, -0.5766],
        [ 0.1624, -0.4310],
        [ 0.2769, -0.6023],
        [ 0.1836, -0.4652],
        [ 0.2013, -0.4929],
        [ 0.2789, -0.6043],
        [ 0.2807, -0.

Output: tensor([[ 0.1995, -0.4403],
        [ 0.2459, -0.5054],
        [ 0.1763, -0.4101],
        [ 0.1765, -0.4116],
        [ 0.1355, -0.3492],
        [ 0.1843, -0.4219],
        [ 0.1545, -0.3790],
        [ 0.1980, -0.4386],
        [ 0.2262, -0.4815],
        [ 0.1711, -0.4034],
        [ 0.1852, -0.4226],
        [ 0.1753, -0.4071],
        [ 0.2018, -0.4439],
        [ 0.1615, -0.3870],
        [ 0.1794, -0.4157],
        [ 0.1946, -0.4336],
        [ 0.2371, -0.4908],
        [ 0.1982, -0.4395],
        [ 0.1985, -0.4400],
        [ 0.1970, -0.4377],
        [ 0.2162, -0.4662],
        [ 0.1832, -0.4171],
        [ 0.1715, -0.4031],
        [ 0.1730, -0.4008],
        [ 0.2435, -0.5039],
        [ 0.1938, -0.4341],
        [ 0.1678, -0.3941],
        [ 0.0986, -0.2999],
        [ 0.1736, -0.4039],
        [ 0.2013, -0.4435],
        [ 0.1853, -0.4218],
        [ 0.1997, -0.4406],
        [ 0.1779, -0.4106],
        [ 0.1464, -0.3646],
        [ 0.1787, -0.4125],
        [ 0.

...

Loss:  tensor(0.5622, grad_fn=<NllLossBackward>)
Output: tensor([[ 0.2118, -0.3519],
        [ 0.4358, -0.6271],
        [ 0.2096, -0.3549],
        [ 0.2025, -0.3380],
        [ 0.3037, -0.4560],
        [ 0.2166, -0.3603],
        [ 0.2257, -0.3743],
        [ 0.2055, -0.3451],
        [ 0.3322, -0.5002],
        [ 0.2482, -0.4066],
        [ 0.2203, -0.3663],
        [ 0.2603, -0.4173],
        [ 0.2002, -0.3378],
        [ 0.2291, -0.3774],
        [ 0.2460, -0.3977],
        [ 0.2021, -0.3406],
        [ 0.3947, -0.5717],
        [ 0.2227, -0.3680],
        [ 0.3238, -0.4901],
        [ 0.2046, -0.3437],
        [ 0.2580, -0.4125],
        [ 0.2176, -0.3622],
        [ 0.2066, -0.3511],
        [ 0.3243, -0.4933],
        [ 0.4224, -0.6097],
        [ 0.2168, -0.3605],
        [ 0.3816, -0.5679],
        [ 0.1417, -0.2739],
        [ 0.2823, -0.4421],
        [ 0.2044, -0.3438],
        [ 0.2020, -0.3419],
        [ 0.2020, -0.3398],
        [ 0.2125, -0.3568],
        [ 0.1762, -

Loss:  tensor(0.5376, grad_fn=<NllLossBackward>)
Output: tensor([[ 0.4955, -0.7774],
        [ 0.3689, -0.6241],
        [ 0.3616, -0.6104],
        [ 0.3315, -0.5700],
        [ 0.3504, -0.5958],
        [ 0.3578, -0.6056],
        [ 0.4054, -0.6646],
        [ 0.4418, -0.7079],
        [ 0.3764, -0.6309],
        [ 0.3577, -0.6051],
        [ 0.3308, -0.5802],
        [ 0.3881, -0.6427],
        [ 0.5301, -0.8203],
        [ 0.3613, -0.6074],
        [ 0.3661, -0.6154],
        [ 0.3496, -0.6012],
        [ 0.4487, -0.7223],
        [ 0.3615, -0.6137],
        [ 0.3148, -0.5554],
        [ 0.4473, -0.7156],
        [ 0.5214, -0.8082],
        [ 0.3476, -0.5915],
        [ 0.4934, -0.7693],
        [ 0.3552, -0.6015],
        [ 0.3604, -0.6087],
        [ 0.3958, -0.6530],
        [ 0.3769, -0.6291],
        [ 0.3592, -0.6072],
        [ 0.3297, -0.5663],
        [ 0.1922, -0.3897],
        [ 0.3420, -0.5784],
        [ 0.4134, -0.6743],
        [ 0.5291, -0.8195],
        [ 0.4549, -

torch.Size([90, 20, 375])




Traceback (most recent call last):
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

### Baseline

In [ ]:
heldout = int(len(dataset) * 0.2)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - heldout, heldout])

model = Baseline(add_brnn=False)
config = dict(
    num_workers=8,
    batch_size=90,
    learning_rate=0.001,
    weight_decay=0.01,
    
    num_epochs=200,

#     num_epochs=200,
    is_notebook=True
)

train(model, train_dataset, config)

## Testing

In [ ]:
y_true = dataset.labels[test_dataset.indices]
y_pred, test_acc = test(model, test_dataset, config)
print(len(test_dataset))
print(dataset.labels[test_dataset.indices].sum().item())

In [ ]:
results = pd.DataFrame(classification_report(y_true, y_pred, zero_division=0, output_dict=True)).transpose()
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
specificity = tn / (tn+fp)
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)
auc_score = metrics.auc(fpr, tpr)

In [ ]:
display(results)
print("Specificity:", specificity)
print("AUC:", auc_score)

In [ ]:
lr_auc = roc_auc_score(y_true, y_pred)
print('ROC AUC=%.3f' % (lr_auc))
lr_fpr, lr_tpr, _ = roc_curve(y_true, y_pred)
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Baseline model')

pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
pyplot.legend()
pyplot.show()

In [ ]:
pr_auc = average_precision_score(y_true, y_pred)
print("PR AUC:", specificity)